In [97]:
from __future__ import unicode_literals

path_direction = "resources/transition-data.txt"

In [98]:
import random
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm

datas = [x.strip() for x in tqdm.tqdm(codecs.open(F'{path_direction}','rU','utf-8').readlines())]

100%|██████████| 19/19 [00:00<00:00, 81401.20it/s]


In [99]:
print(datas)

['علی به مدرسه رفت.', 'ما با پیکان جوانان گوجه ای به مشهَدالرضا رفتیم.', 'محمد و دوستش با تیبا از مهاباد به شیراز سفر کردند.', 'فاطمه با ماشین به یزد آمد.', 'اتوبوس به تهران رسید.', 'علی به مقصد اهواز حرکت کرد.', 'خوانواده هاشمی با اتوبوس از نیشایور به قم رفتند.', 'محمد رضا با پراید از ماهشهر به تهران رفت.', 'دوستان من برای اردو به شیراز رفتند.', 'سفر با ماشین به خراسان به پایان رسید.', 'نیسان آبی از ارومیه به تبریز رسید.', 'سفر با هواپیما از تهران به شیراز آسان بود.', 'فردا با پژو به مشهد خواهیم رفت.', 'ما با تاکسی به کرج رفتیم.', 'دوستم با هواپیما به یزد سفر کرد.', 'دوستان من با سمند به کرج رفتند.', 'اتوبوس بازیکنان به رست رسید.', 'مسافران قطار از تهران به مقصد شهر ری سوار شدند.', 'دیروز با هواپیما به تهران آمدم.']


In [100]:

from hazm import *

normalizer = Normalizer()

data_normalized = [[normalizer.normalize(y) for y in x] for x in tqdm.tqdm(datas)]

100%|██████████| 19/19 [00:00<00:00, 1352.45it/s]


In [101]:
for x in data_normalized :
    print(''.join(x))

علی به مدرسه رفت.
ما با پیکان جوانان گوجه ای به مشهدالرضا رفتیم.
محمد و دوستش با تیبا از مهاباد به شیراز سفر کردند.
فاطمه با ماشین به یزد آمد.
اتوبوس به تهران رسید.
علی به مقصد اهواز حرکت کرد.
خوانواده هاشمی با اتوبوس از نیشایور به قم رفتند.
محمد رضا با پراید از ماهشهر به تهران رفت.
دوستان من برای اردو به شیراز رفتند.
سفر با ماشین به خراسان به پایان رسید.
نیسان آبی از ارومیه به تبریز رسید.
سفر با هواپیما از تهران به شیراز آسان بود.
فردا با پژو به مشهد خواهیم رفت.
ما با تاکسی به کرج رفتیم.
دوستم با هواپیما به یزد سفر کرد.
دوستان من با سمند به کرج رفتند.
اتوبوس بازیکنان به رست رسید.
مسافران قطار از تهران به مقصد شهر ری سوار شدند.
دیروز با هواپیما به تهران آمدم.


In [102]:
data_sentences = [sent_tokenize(''.join(x)) for x in tqdm.tqdm(data_normalized)]

100%|██████████| 19/19 [00:00<00:00, 19101.58it/s]


In [103]:
for x in data_sentences:
    print('###'.join(x))

علی به مدرسه رفت.
ما با پیکان جوانان گوجه ای به مشهدالرضا رفتیم.
محمد و دوستش با تیبا از مهاباد به شیراز سفر کردند.
فاطمه با ماشین به یزد آمد.
اتوبوس به تهران رسید.
علی به مقصد اهواز حرکت کرد.
خوانواده هاشمی با اتوبوس از نیشایور به قم رفتند.
محمد رضا با پراید از ماهشهر به تهران رفت.
دوستان من برای اردو به شیراز رفتند.
سفر با ماشین به خراسان به پایان رسید.
نیسان آبی از ارومیه به تبریز رسید.
سفر با هواپیما از تهران به شیراز آسان بود.
فردا با پژو به مشهد خواهیم رفت.
ما با تاکسی به کرج رفتیم.
دوستم با هواپیما به یزد سفر کرد.
دوستان من با سمند به کرج رفتند.
اتوبوس بازیکنان به رست رسید.
مسافران قطار از تهران به مقصد شهر ری سوار شدند.
دیروز با هواپیما به تهران آمدم.


In [104]:
data_tokens = [[word_tokenize(sentence) for sentence in sentences] for sentences in tqdm.tqdm(data_sentences)]

100%|██████████| 19/19 [00:00<00:00, 15222.88it/s]


In [105]:
from nltk import FreqDist
import itertools
import pandas as pd

all_tokens = list(itertools.chain(*itertools.chain(*data_tokens)))

dataframe = {}

for opt in ['all']:
    dataframe[opt] = FreqDist(eval(F"{opt}_tokens")).most_common(10)

freq_anlyst = pd.DataFrame(dataframe)

In [106]:
freq_anlyst

,all
0,"(به, 20)"
1,"(., 19)"
2,"(با, 12)"
3,"(از, 6)"
4,"(تهران, 5)"
5,"(سفر, 4)"
6,"(رسید, 4)"
7,"(شیراز, 3)"
8,"(اتوبوس, 3)"
9,"(رفتند, 3)"


In [107]:
print ('%-16s' % 'Number of words', '%-16s' % len(all_tokens))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(all_tokens)))
avg=np.sum([len(word) for word in all_tokens])/len(all_tokens)
print ('%-16s' % 'Average word length', '%-16s' % avg)
print ('%-16s' % 'Longest word', '%-16s' % all_tokens[np.argmax([len(word) for word in all_tokens])])

Number of words  155             
Number of unique words 72              
Average word length 3.432258064516129
Longest word     خواهیم_رفت      


In [108]:
stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('resources/stopwords.txt','r','utf-8').readlines()]

In [109]:
all_tokens_nonstop = [t for t in tqdm.tqdm(all_tokens) if t not in stopwords]

dataframe_nonstop = {}

for opt in ['all']:
     dataframe_nonstop[opt] = FreqDist(eval(F"{opt}_tokens_nonstop")).most_common(30)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)

100%|██████████| 155/155 [00:00<00:00, 70199.45it/s]


In [110]:
freq_analysis_nonstop

,all
0,"(., 19)"
1,"(تهران, 5)"
2,"(سفر, 4)"
3,"(شیراز, 3)"
4,"(اتوبوس, 3)"
5,"(رفتند, 3)"
6,"(هواپیما, 3)"
7,"(علی, 2)"
8,"(رفتیم, 2)"
9,"(محمد, 2)"


In [111]:
stemmer = Stemmer()
lemmatizer = Lemmatizer()

def get_lemma_set(token,opt=1):
    if opt == 1 :
        return stemmer.stem(token)
    elif opt ==2 :
        return lemmatizer.lemmatize(token)

In [112]:
opt = 2

all_tokens_nonstop_stem = [get_lemma_set(t, opt) for t in tqdm.tqdm(all_tokens_nonstop)]

dataframe_nonstop_stem = {}

for opt in ['stem']:
     dataframe_nonstop_stem[opt] = FreqDist(eval("all_tokens_nonstop_stem")).most_common(30)

freq_analysis_nonstop_stem = pd.DataFrame(dataframe_nonstop_stem)
freq_analysis_nonstop_stem

100%|██████████| 99/99 [00:00<00:00, 100492.76it/s]


,stem
0,"(., 19)"
1,"(رفت#رو, 6)"
2,"(تهران, 5)"
3,"(سفر, 4)"
4,"(شیراز, 3)"
5,"(اتوبوس, 3)"
6,"(هواپیما, 3)"
7,"(علی, 2)"
8,"(محمد, 2)"
9,"(ماشین, 2)"


In [113]:
from wapiti import Model
tagger = POSTagger(model = 'resources/postagger.model')

ModuleNotFoundError: No module named 'wapiti'

In [114]:
import copy
dataframe_nonstop_stem_advjj = {}

selected=[]
for x,y in FreqDist(eval("all_tokens_nonstop_stem")).most_common(30):
    pos = tagger.tag([x])[0][1]
    if pos in ['adj']:
        selected.append((x,y))
dataframe_nonstop_stem_advjj = copy.deepcopy(selected[0:40])
dataframe_nonstop_stem_advjj = pd.DataFrame(dataframe_nonstop_stem_advjj)

NameError: name 'tagger' is not defined

In [115]:
dataframe_nonstop_stem_advjj

{}